# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

# 语言翻译
在这个项目中，您将要窥探神经网络机器翻译的领域。 您将在英文和法文句子的数据集上训练一个序列来对序列模型进行排序，这可以将新句子从英语翻译成法语。
## 获取数据
因为将英语的整个语言翻译成法语将需要很多时间来训练，我们为您提供了英文语料库的一小部分。

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [14]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.
## 实现预处理功能
### 文本到Word Ids
与其他RNN一样，您必须将文本转换成数字，以便计算机能够理解。 在函数`text_to_ids（）`中，你可以将`source_text`和`target_text`从单词转成ids。 但是，您需要在`target_text'的末尾添加`<EOS>`word id。 这将有助于神经网络预测句子何时结束。

您可以通过以下方式获取“<EOS>”字id：
```蟒蛇
target_vocab_to_int[ '<EOS>']
```
您可以使用“source_vocab_to_int”和“target_vocab_to_int”获取其他单词id。

In [15]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    将源文本和目标文本转换为正确的字ids
     ：param source_text：包含所有源文本的字符串。
     ：param target_text：包含所有目标文本的字符串。
     ：param source_vocab_to_int：从源字词到id的字典
     ：param target_vocab_to_int：从目标词到id的字典
     ：return：列表元组（source_id_text，target_id_text）
    """
    #定义源文件ID
    source_text_id = []
    #使用换行分割文本
    for sentence in source_text.split("\n"):
        #使用空格分割句子，转换成词典中某个单词对应的数字
        sentence_id =[source_vocab_to_int[word] for word in sentence.split()]
        source_text_id.append(sentence_id)
    
    #和上面的实现一样
    target_text_id=[]
    for sentence in target_text.split("\n"):
        sentence_id =[target_vocab_to_int[word] for word in sentence.split()]+ [target_vocab_to_int['<EOS>']] 
        target_text_id.append(sentence_id)
    print (source_text_id)
    print (target_text_id)
    return (source_text_id, target_text_id)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

[[18, 17, 15, 26, 19, 20, 4, 11, 30, 32, 15, 27, 16, 6, 7], [22, 13, 23, 15, 28, 9, 20, 5, 11, 30, 32, 15, 28, 8, 16, 10, 7], [21, 15, 28, 19, 20, 31, 11, 30, 32, 15, 28, 29, 16, 25, 7], [22, 13, 23, 15, 26, 24, 20, 25, 11, 30, 32, 15, 14, 16, 12, 7]]
[[16, 15, 17, 27, 12, 11, 26, 18, 8, 32, 30, 17, 28, 25, 21, 5, 1], [10, 7, 17, 23, 4, 25, 19, 8, 32, 30, 29, 14, 25, 31, 5, 1], [20, 17, 23, 12, 25, 13, 8, 32, 30, 17, 23, 6, 25, 22, 5, 1], [10, 7, 17, 27, 33, 25, 22, 8, 32, 30, 9, 4, 25, 24, 5, 1]]
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.
### 预处理所有数据并保存
运行下面的代码单元将预处理所有的数据并保存到文件。

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.
# 检查点
这是你的第一个检查点。 如果你决定回到笔记本电脑或者重新启动笔记本电脑，你可以从这里开始。 预处理数据已保存到磁盘。

In [54]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
print (source_int_text[0])
print (source_vocab_to_int)

[37, 197, 87, 163, 38, 149, 63, 71, 55, 60, 87, 104, 35, 6, 7]
{'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3, 'july': 4, 'sharks': 5, 'april': 6, '.': 7, 'been': 8, 'strawberries.': 9, 'december': 10, 'lemon': 11, 'her': 12, 'apples': 13, 'they': 14, 'oranges': 15, 'animals': 16, 'would': 17, 'pears': 18, 'dislike': 19, 'busy': 20, 'grapes.': 21, 'strawberry.': 22, 'lemons.': 23, 'fall': 24, 'horse': 25, 'cats': 26, 'orange': 27, 'am': 28, 'august': 29, 'elephant': 30, 'translating': 31, 'was': 32, 'where': 33, 'dogs': 34, 'in': 35, 'apple': 36, 'new': 37, 'quiet': 38, 'lemon.': 39, 'india': 40, 'lions': 41, 'lake': 42, 'elephants': 43, 'why': 44, 'i': 45, 'june': 46, 'our': 47, 'nice': 48, 'did': 49, 'liked': 50, 'shiny': 51, 'rabbit': 52, 'go': 53, 'winter': 54, 'and': 55, 'a': 56, 'monkey': 57, 'between': 58, 'think': 59, 'it': 60, 'limes.': 61, 'has': 62, 'autumn': 63, 'peaches': 64, 'but': 65, 'how': 66, 'plan': 67, 'dry': 68, 'never': 69, 'february': 70, ',': 71, 'weather': 72, 

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

### 检查TensorFlow版本和访问GPU
这将检查以确保您具有正确版本的TensorFlow和访问GPU

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1


/Users/kangfei/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

## 建立神经网络
您将通过实现以下功能构建构建序列到序列模型所必需的组件：
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decode_layer_train`
- `decode_layer_infer`
- `decode_layer`
- `seq2seq_model`

### 输入
实现`model_inputs（）`函数来创建神经网络的TF占位符。应该创建以下占位符：

- 使用名称为“2”的TF占位符名称参数输入名为“input”的文本占位符。
- 将占位符排在第2位。
- 等级为0的学习率占位符。
- 使用名为“0”的TF占位符名称参数保留名为“keep_prob”的概率占位符。
- 具有等级1的名为“target_sequence_length”的目标序列长度占位符
- 命名为“max_target_len”的最大目标序列长度张量值从target_sequence_length占位符上应用tf.reduce_max获取其值。排名0。
- 源序列长度占位符名为“source_sequence_length”等级1

返回占位符在下面的元组（输入，目标，学习率，保持概率，目标序列长度，最大目标序列长度，源序列长度）

In [28]:
def model_inputs():
    """
    创建TF占位符，以获取输入，目标，学习速率以及源和目标序列的长度。
     ：返回：元组（输入，目标，学习率，保持概率，目标序列长度，
     最大目标序列长度，源序列长度）
     
    tf.placeholder(dtype, shape=None, name=None)
    此函数可以理解为形参，用于定义过程，在执行的时候再赋具体的值
    参数：
    dtype：数据类型。常用的是tf.float32,tf.float64等数值类型
    shape：数据形状。默认是None，就是一维值，也可以是多维，比如[2,3], [None, 3]表示列是3，行不定
    name：名称。
    """
    #输入的数据，定义占位符，2维的输入。 第一个None：输入的batch_size；第二个None：一句话的长度，因为不确定，所以仍旧是none
    input_data = tf.placeholder(tf.int32,shape=[None,None],name='input')
    #输出的目标
    targets = tf.placeholder(tf.int32,shape=[None,None],name='targets')
    #学习率的定义，
    learning_rate = tf.placeholder(tf.float32,shape=[],name='learning_rate')
    #drop丢弃的占位符
    keep_prob = tf.placeholder(tf.float32,shape=[],name='keep_prob')
    #目标序列的长度
    target_sequence_length = tf.placeholder(tf.int32,[None,],name='target_sequence_length')
    #最大目标序列长度
    max_target_length = tf.reduce_max(target_sequence_length)
    #源序列长度
    source_sequence_length = tf.placeholder(tf.int32,[None,],name='source_sequence_length')
    
    return (input_data,targets,learning_rate,keep_prob,target_sequence_length,max_target_length,source_sequence_length)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.
### 过程解码器输入
通过删除`target_data`中每个批次的最后一个字id并将GO ID连接到每个批次的开始来实现`process_decoder_input`。

In [57]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    用于编码的预处理目标数据
     ：param target_data：Target Placehoder
     ：param target_vocab_to_int：从目标词到id的字典
     ：param batch_size：批量大小
     ：返回：预处理的目标数据
     tf.strided_slice函数说明：
    来把输入变个型,可以看成3维的tensor,从外向为1,2,3维
    以tf.strided_slice(input, [0,0,0], [2,2,2], [1,2,1])调用为例,start = [0,0,0] , end = [2,2,2], stride = [1,2,1],求一个[start, end)的一个片段,注意end为开区间
    第1维 start = 0 , end = 2, stride = 1, 所以取 0 , 1行，此时的输出
    output1=
    [
         [
              [1,1,1]
              [2,2,2]
         ]
         [
             [3,3,3]
             [4,4,4]
         ]
    ]
    第2维时, start = 0 , end = 2 , stride = 2, 所以只能取0行,此时的输出
    output2=
    [
         [
              [1,1,1]
         ]
         [
             [3,3,3]
         ]
    ]
    第3维的时候,start = 0, end = 2, stride = 1, 可以取0,1行,此时得到的就是最后的输出
    [
         [
              [1,1]
         ]
         [
             [3,3]
         ]
    ]
    整理之后最终的输出为:
    [[[1,1],[3,3]]]
    
    
    函数原型 tf.concat(concat_dim,values,name='concat')
   参数:
      concat_dim：值范围是[0,rank(values)-1]。等于0，表示沿着第一维连接tensor；等于1，表示沿着第二维度连接tensor，……
      values:tensor值列表

   例子：
    x=tf.constant([[1,2,3],[4,5,6]])
    y=tf.constant([[7,8,9,10],[11,12,13,14]])
    tf.concat(0,[x,y])=>[[1,2,3],[4,5,6],[7,8,9,10],[11,12,13,14]]
    tf.concat(1,[x,y])=>[[1,2,3,7,8,9,10],[4,5,6,11,12,13,14]]
    
    """
    ending = tf.strided_slice(target_data,[0,0],[batch_size,-1],[1,1])
    print(ending)
    return   tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tensor("StridedSlice:0", shape=(2, 2), dtype=int32)
Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 
 ### 编码
实现`encoding_layer（）`来创建一个编码器RNN层：
  *使用[`tf.contrib.layers.embed_sequence`]嵌入编码器输入（https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence ）
  *构造一个[堆叠]（https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms ）[`tf.contrib.rnn.LSTMCell`] （https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell ）包装在一个[`tf.contrib.rnn.DropoutWrapper`]（https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper ）
  *将单元格和嵌入式输入传递给[`tf.nn.dynamic_rnn（）`]（https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn ）

In [58]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    创建编码层
     ：param rnn_inputs：RNN的输入
     ：param rnn_size：RNN大小
     ：param num_layers：层数
     ：param keep_prob：Dropout保持概率
     ：param source_sequence_length：批处理中每个序列的长度列表
     ：param source_vocab_size：源数据的词汇大小
     ：param encoding_embedding_size：嵌入源数据的大小
     ：return：tuple（RNN输出，RNN状态）
     
     [("Garbage piles in the city","Garbage"), 
      ("City is clogged with vehicles","Traffic")]

    我想把每个元组的第一个元素作为一系列单词。这些单词需要以矢量形式嵌入。作为第一步，它们应该被转换为指数或数字。例如，在这种情况下，词汇将是

    [{Garbage:1},{piles:2},{in:3},{the:4},{city:5},{is:6},{clogged:7},{with:8},{vehicles:9}]。

    编码的文本将如下所示：

    features=[[1 2 3 4 5],[5 6 7 8 9]].

    你features可以在批处理中传递这个编码的文本 - 
    tf.contrib.layers.embed_sequence（features，vocab_size = n_words，embed_dim = EMBEDDING_SIZE，scope ='words'）`

    现在，使用索引（1到5）表示的每个单词都嵌入到EMBEDDING_SIZE的向量中。

    如果批量大小为2（即一批中的2个序列），则在我们的情况下，EMBEDDING_SIZE为10，输出将为2×5×10维度的矩阵。

    样品输出 -  [[[0.1,0.3,0.4,0.2,0.5,0.2,0.2,0.2,0.4,0.1],[0.1,0.3,0.4,0.2,0.5,0.2,0.1,0.2,0.4,0.1],[0.1,0.3,0.4,0.2,0.5,0.2,0.4,0.2,0.4,0.1],[0.1,0.3,0.4,0.2,0.5,0.3,0.1,0.2,0.4,0.1],[0.1,0.3,0.4,0.2,0.5,0.2,0.1,0.2,0.4,0.6]],[sent2]]

    sent2 被类似地编码（5×10矩阵）。
    """
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # 定义基本lstm单元
    def create_cell(run_size):
        enc_cell = tf.contrib.rnn.LSTMCell(run_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        return enc_cell
    
    #使用MultiRNNCell 接口连接多层lstm， 
    enc_cell = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size) for _ in range(num_layers)])
    """tensorflow 的dynamic_rnn方法，我们用一个小例子来说明其用法，假设你的RNN的输入input是[2,20,128]，
    其中2是batch_size,20是文本最大长度，128是embedding_size，可以看出，有两个example，我们假设第二个文本长度只有13，
    剩下的7个是使用0-padding方法填充的。dynamic返回的是两个参数：outputs,last_states，其中outputs是[2,20,128]，
    也就是每一个迭代隐状态的输出,last_states是由(c,h)组成的tuple，均为[batch,128]。
    到这里并没有什么不同，但是dynamic有个参数：sequence_length，这个参数用来指定每个example的长度，
    比如上面的例子中，我们令 sequence_length为[20,13]，表示第一个example有效长度为20，第二个example有效长度为13，
    当我们传入这个参数的时候，对于第二个example，TensorFlow对于13以后的padding就不计算了，
    其last_states将重复第13步的last_states直至第20步，而outputs中超过13步的结果将会被置零。
    
    tf.nn.rnn创建固定RNN长度的展开图。 这意味着，如果您使用200个步骤的输入调用tf.nn.rnn，则创建一个具有200个RNN步骤的静态图。 
    首先，图形创建速度很慢。 第二，你不能比原来指定的更长的序列（> 200）.tf.nn.dynamic_rnn解决了这个问题。 
    它使用tf.While循环在执行时动态构建图形。 这意味着图形创建速度更快，您可以提供可变大小的批量。
    """
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, 
                                              sequence_length=source_sequence_length, 
                                              dtype=tf.float32)
    return (enc_output,enc_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)
### 解码 - 训练
创建训练解码层：
*创建一个[`tf.contrib.seq2seq.TrainingHelper`]（https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper ）
*创建一个[`tf.contrib.seq2seq.BasicDecoder`]（ https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder ）
*从[`tf.contrib.seq2seq.dynamic_decode`]（https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode ）获取解码器输出

In [36]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    创建一个解码层进行训练
     ：param encoder_state：编码器状态
     ：param dec_cell：Decoder RNN Cell
     ：param dec_embed_input：解码器嵌入式输入
     ：param target_sequence_length：目标批次中每个序列的长度
     ：param max_summary_length：批次中最长序列的长度
     ：param output_layer：应用输出层的功能
     ：param keep_prob：Dropout保持概率
     ：return：BasicDecoderOutput包含训练逻辑和sample_id
     
     tf-seq2seq 是一个用于 TensorFlow 的通用编码器-解码器框架（encoder-decoder framework），
     其可用于机器翻译、文本摘要、会话建模、图像描述等任务。
    """
    #TrainingHelper用于train阶段，next_inputs方法一样也接收outputs与sample_ids，但是只是从初始化时的inputs返回下一时刻的输入。
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,sequence_length=target_sequence_length,time_major=False)
    
    training_decoder =tf.contrib.seq2seq.BasicDecoder(dec_cell,training_helper,encoder_state,output_layer)
    
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,impute_finished=True,maximum_iterations=max_summary_length)[0]
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [37]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_tokens  = tf.tile(tf.constant([start_of_sequence_id],dtype=tf.int32),[batch_size],name="start_tokens")
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,start_tokens,end_of_sequence_id)
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,inference_helper,encoder_state,output_layer)
    
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,impute_finished=True,maximum_iterations=max_target_sequence_length)[0]
    
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.
### 构建解码层
实现`decode_layer（）`来创建一个解码器RNN层。

*嵌入目标序列
*构造解码器LSTM单元（就像你在上面构建编码器单元）
*创建输出层，将解码器的输出映射到我们的词汇表的元素
*使用你的`decode_layer_train（encoder_state，dec_cell，dec_embed_input，target_sequence_length，max_target_sequence_length，output_layer，keep_prob）`函数来获取训练逻辑。
*使用`decode_layer_infer（encoder_state，dec_cell，dec_embeddings，start_of_sequence_id，end_of_sequence_id，max_target_sequence_length，vocab_size，output_layer，batch_size，keep_prob）`函数来获取推理逻辑。

注意：您需要使用[tf.variable_scope]（https://www.tensorflow.org/api_docs/python/tf/variable_scope ）在训练和推理之间共享变量。

In [38]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    
    创建解码层
     ：param dec_input：解码输入
     ：param encoder_state：编码器状态
     ：param target_sequence_length：目标批次中每个序列的长度
     ：param max_target_sequence_length：目标序列的最大长度
     ：param rnn_size：RNN大小
     ：param num_layers：层数
     ：param target_vocab_to_int：从目标词到id的字典
     ：param target_vocab_size：目标词汇的大小
     ：param batch_size：批量的大小
     ：param keep_prob：Dropout保持概率
     ：param decode_embedding_size：解码嵌入大小
     ：return：Tuple of（Training BasicDecoderOutput，Inference BasicDecoderOutput）
    """
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(
            encoder_state, dec_cell, dec_embed_input, target_sequence_length, 
            max_target_sequence_length, output_layer, keep_prob
        )
        
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_output = decoding_layer_infer(
            encoder_state, dec_cell, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], max_target_sequence_length,
            target_vocab_size, output_layer, batch_size, keep_prob)
         

    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

### 建立神经网络
将上述实现的功能应用于：

- 使用`encoding_layer（rnn_inputs，rnn_size，num_layers，keep_prob，source_sequence_length，source_vocab_size，encoding_embedding_size））编码输入。
- 使用'process_decoder_input（target_data，target_vocab_to_int，batch_size）'函数处理目标数据。
- 使用`decode_layer（dec_input，enc_state，target_sequence_length，max_target_sentence_length，rnn_size，num_layers，target_vocab_to_int，target_vocab_size，batch_size，keep_prob，dec_embedding_size）`函数解码编码输入。

In [40]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    
    构建神经网络的序列到序列部分
     ：param input_data：输入占位符
     ：param target_data：目标占位符
     ：param keep_prob：Dropout保留概率占位符
     ：param batch_size：批量大小
     ：param source_sequence_length：序列批次中源序列的长度
     ：param target_sequence_length：序列批次中目标序列的长度
     ：param source_vocab_size：源词汇大小
     ：param target_vocab_size：目标词汇大小
     ：param enc_embedding_size：解码器嵌入大小
     ：param dec_embedding_size：编码器嵌入大小
     ：param rnn_size：RNN大小
     ：param num_layers：层数
     ：param target_vocab_to_int：从目标词到id的字典
     ：return：Tuple of（Training BasicDecoderOutput，Inference BasicDecoderOutput）
    """
    _,enc_state = encoding_layer(input_data,rnn_size,num_layers,keep_prob,source_sequence_length
                                ,source_vocab_size,enc_embedding_size)
    
    dec_input = process_decoder_input(target_data,target_vocab_to_int,batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(
        dec_input, enc_state, target_sequence_length, max_target_sentence_length, 
        rnn_size, num_layers, target_vocab_to_int, target_vocab_size, 
        batch_size, keep_prob, dec_embedding_size
    )
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

## 神经网络训练
### 超参数
调整以下参数：

- 将“epochs”设定为历元的数量。
- 将batch_size设置为批量大小。
- 将“rnn_size”设置为RNN的大小。
- 将`num_layers'设置为层数。
- 将“learning_rate”设置为学习率。
- 将`keep_probability`设置为Dropout保持概率
- 设置`display_step`以显示每个调试输出语句之间的步数

In [41]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 20
decoding_embedding_size = 20
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.8
display_step = 20

### Build the Graph
Build the graph using the neural network you implemented.

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   20/1077 - Train Accuracy: 0.2824, Validation Accuracy: 0.3512, Loss: 3.2397
Epoch   0 Batch   40/1077 - Train Accuracy: 0.3395, Validation Accuracy: 0.4031, Loss: 2.9971
Epoch   0 Batch   60/1077 - Train Accuracy: 0.4167, Validation Accuracy: 0.4560, Loss: 2.6101
Epoch   0 Batch   80/1077 - Train Accuracy: 0.4156, Validation Accuracy: 0.4719, Loss: 2.4863
Epoch   0 Batch  100/1077 - Train Accuracy: 0.4520, Validation Accuracy: 0.5043, Loss: 2.3663
Epoch   0 Batch  120/1077 - Train Accuracy: 0.4578, Validation Accuracy: 0.5117, Loss: 2.1874
Epoch   0 Batch  140/1077 - Train Accuracy: 0.4149, Validation Accuracy: 0.4940, Loss: 2.1026
Epoch   0 Batch  160/1077 - Train Accuracy: 0.4461, Validation Accuracy: 0.4904, Loss: 1.7687
Epoch   0 Batch  180/1077 - Train Accuracy: 0.4379, Validation Accuracy: 0.5092, Loss: 1.6118
Epoch   0 Batch  200/1077 - Train Accuracy: 0.4184, Validation Accuracy: 0.4538, Loss: 1.5052
Epoch   0 Batch  220/1077 - Train Accuracy: 0.3873, Validati

Epoch   1 Batch  720/1077 - Train Accuracy: 0.7874, Validation Accuracy: 0.7695, Loss: 0.2558
Epoch   1 Batch  740/1077 - Train Accuracy: 0.8246, Validation Accuracy: 0.8011, Loss: 0.2312
Epoch   1 Batch  760/1077 - Train Accuracy: 0.8395, Validation Accuracy: 0.8100, Loss: 0.2333
Epoch   1 Batch  780/1077 - Train Accuracy: 0.8012, Validation Accuracy: 0.8001, Loss: 0.2529
Epoch   1 Batch  800/1077 - Train Accuracy: 0.8203, Validation Accuracy: 0.7596, Loss: 0.2353
Epoch   1 Batch  820/1077 - Train Accuracy: 0.8199, Validation Accuracy: 0.8004, Loss: 0.2340
Epoch   1 Batch  840/1077 - Train Accuracy: 0.8223, Validation Accuracy: 0.8150, Loss: 0.2191
Epoch   1 Batch  860/1077 - Train Accuracy: 0.8356, Validation Accuracy: 0.8164, Loss: 0.2139
Epoch   1 Batch  880/1077 - Train Accuracy: 0.8477, Validation Accuracy: 0.8104, Loss: 0.2026
Epoch   1 Batch  900/1077 - Train Accuracy: 0.8328, Validation Accuracy: 0.8050, Loss: 0.2085
Epoch   1 Batch  920/1077 - Train Accuracy: 0.8344, Validati

Epoch   3 Batch  360/1077 - Train Accuracy: 0.9164, Validation Accuracy: 0.8789, Loss: 0.0582
Epoch   3 Batch  380/1077 - Train Accuracy: 0.9336, Validation Accuracy: 0.9077, Loss: 0.0570
Epoch   3 Batch  400/1077 - Train Accuracy: 0.9152, Validation Accuracy: 0.8732, Loss: 0.0702
Epoch   3 Batch  420/1077 - Train Accuracy: 0.9336, Validation Accuracy: 0.9119, Loss: 0.0534
Epoch   3 Batch  440/1077 - Train Accuracy: 0.9109, Validation Accuracy: 0.8931, Loss: 0.0711
Epoch   3 Batch  460/1077 - Train Accuracy: 0.9195, Validation Accuracy: 0.9201, Loss: 0.0695
Epoch   3 Batch  480/1077 - Train Accuracy: 0.9042, Validation Accuracy: 0.8878, Loss: 0.0581
Epoch   3 Batch  500/1077 - Train Accuracy: 0.9293, Validation Accuracy: 0.8960, Loss: 0.0508
Epoch   3 Batch  520/1077 - Train Accuracy: 0.9334, Validation Accuracy: 0.8910, Loss: 0.0590
Epoch   3 Batch  540/1077 - Train Accuracy: 0.9160, Validation Accuracy: 0.9023, Loss: 0.0554
Epoch   3 Batch  560/1077 - Train Accuracy: 0.9129, Validati

Epoch   4 Batch 1060/1077 - Train Accuracy: 0.9473, Validation Accuracy: 0.9364, Loss: 0.0344
Epoch   5 Batch   20/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9371, Loss: 0.0329
Epoch   5 Batch   40/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9222, Loss: 0.0331
Epoch   5 Batch   60/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.8931, Loss: 0.0304
Epoch   5 Batch   80/1077 - Train Accuracy: 0.9121, Validation Accuracy: 0.9300, Loss: 0.0388
Epoch   5 Batch  100/1077 - Train Accuracy: 0.9316, Validation Accuracy: 0.9137, Loss: 0.0473
Epoch   5 Batch  120/1077 - Train Accuracy: 0.9672, Validation Accuracy: 0.9165, Loss: 0.0457
Epoch   5 Batch  140/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9027, Loss: 0.0426
Epoch   5 Batch  160/1077 - Train Accuracy: 0.9496, Validation Accuracy: 0.9077, Loss: 0.0343
Epoch   5 Batch  180/1077 - Train Accuracy: 0.9516, Validation Accuracy: 0.9183, Loss: 0.0446
Epoch   5 Batch  200/1077 - Train Accuracy: 0.9285, Validati

Epoch   6 Batch  700/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9460, Loss: 0.0314
Epoch   6 Batch  720/1077 - Train Accuracy: 0.9420, Validation Accuracy: 0.9535, Loss: 0.0380
Epoch   6 Batch  740/1077 - Train Accuracy: 0.9402, Validation Accuracy: 0.9361, Loss: 0.0313
Epoch   6 Batch  760/1077 - Train Accuracy: 0.9477, Validation Accuracy: 0.9428, Loss: 0.0379
Epoch   6 Batch  780/1077 - Train Accuracy: 0.9176, Validation Accuracy: 0.9421, Loss: 0.0501
Epoch   6 Batch  800/1077 - Train Accuracy: 0.9363, Validation Accuracy: 0.9496, Loss: 0.0380
Epoch   6 Batch  820/1077 - Train Accuracy: 0.9418, Validation Accuracy: 0.9382, Loss: 0.0280
Epoch   6 Batch  840/1077 - Train Accuracy: 0.9703, Validation Accuracy: 0.9371, Loss: 0.0342
Epoch   6 Batch  860/1077 - Train Accuracy: 0.9379, Validation Accuracy: 0.9439, Loss: 0.0408
Epoch   6 Batch  880/1077 - Train Accuracy: 0.9645, Validation Accuracy: 0.9368, Loss: 0.0305
Epoch   6 Batch  900/1077 - Train Accuracy: 0.9355, Validati

Epoch   8 Batch  340/1077 - Train Accuracy: 0.9692, Validation Accuracy: 0.9343, Loss: 0.0312
Epoch   8 Batch  360/1077 - Train Accuracy: 0.9512, Validation Accuracy: 0.9254, Loss: 0.0323
Epoch   8 Batch  380/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9521, Loss: 0.0253
Epoch   8 Batch  400/1077 - Train Accuracy: 0.9367, Validation Accuracy: 0.9339, Loss: 0.0362
Epoch   8 Batch  420/1077 - Train Accuracy: 0.9633, Validation Accuracy: 0.9258, Loss: 0.0250
Epoch   8 Batch  440/1077 - Train Accuracy: 0.9223, Validation Accuracy: 0.9286, Loss: 0.0402
Epoch   8 Batch  460/1077 - Train Accuracy: 0.9410, Validation Accuracy: 0.9595, Loss: 0.0379
Epoch   8 Batch  480/1077 - Train Accuracy: 0.9502, Validation Accuracy: 0.9375, Loss: 0.0248
Epoch   8 Batch  500/1077 - Train Accuracy: 0.9551, Validation Accuracy: 0.9474, Loss: 0.0219
Epoch   8 Batch  520/1077 - Train Accuracy: 0.9695, Validation Accuracy: 0.9347, Loss: 0.0304
Epoch   8 Batch  540/1077 - Train Accuracy: 0.9453, Validati

Epoch   9 Batch 1040/1077 - Train Accuracy: 0.9396, Validation Accuracy: 0.9425, Loss: 0.0425
Epoch   9 Batch 1060/1077 - Train Accuracy: 0.9707, Validation Accuracy: 0.9425, Loss: 0.0296
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [45]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [48]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    word_ids = [vocab_to_int.get(word, vocab_to_int['<UNK>']) 
                for word in sentence.split()]
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [49]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [84, 125, 56, 214, 200, 141, 7]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [324, 52, 52, 20, 323, 114, 91, 323, 114, 46, 339, 323, 6, 1]
  French Words: il pas pas le pamplemousse , mais pamplemousse , et les pamplemousse . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.

## 不完美的翻译
你可能会注意到一些句子比别人翻译得更好。由于您使用的数据集只有您使用的数千个英文单词的词汇量，您只能使用这些单词看到很好的结果。对于这个项目，你不需要完美的翻译。但是，如果要创建更好的翻译模型，则需要更好的数据。

您可以在[WMT10法语 - 英语语料库]（http://www.statmt.org/wmt10/training-giga-fren.tar ）上训练。这个数据集有更多的词汇和更丰富的话题讨论。然而，这将需要你几天的训练，所以确保你有一个GPU，神经网络在我们提供的数据集上表现良好。在您提交此项目后，请确保您使用WMT10语料库。
## 提交本项目
提交此项目时，请确保在保存笔记本计算机之前运行所有单元格。将笔记本文件保存为“dlnd_language_translation.ipynb”，并将其另存为“File” - >“Download as”下的HTML文件。在您的提交中包含“helper.py”和“problem_unittests.py”文件。